## Creating Classes

In [53]:
from enum import Enum

# Create Enum class with variables
class Score(Enum):
    POSITIVE = 'POSITIVE'
    NEGATIVE = 'NEGATIVE'
    NEUTRAL = 'NEUTRAL'

class Review:
    
    def __init__(self, text, rating):
        self.text = text
        self.rating = rating
        self.feedback = self.get_feedback()
    
    # return positive, neutral, or negative based on rating
    def get_feedback(self):
        if self.rating >= 4:
            return Score.POSITIVE
        elif self.rating == 3:
            return Score.NEUTRAL
        else:
            return Score.NEGATIVE

# Create a sub-class that scales all results proportionally
class ReviewProp(Review):
    
    def __init__(self, text, rating):
        super.()__int__(self, text, rating)
    
    def data_scaling()

## Importing Json

In [64]:
import json
import gzip


filename = 'reviews_Toys_and_Games_5.json.gz'  # Sample file.

reviewText = []
with gzip.open(filename , 'rb') as gzip_file:
    for line in gzip_file:  # Read one line.
        line = line.rstrip()
        if line:  # Any JSON data on it?
            review = json.loads(line)
            reviewText.append(Review(review['reviewText'], review['overall'])) # Appending only comments and scores


## Preparing Data

### Transform comments into vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer



### Splitting Data into Training and Testing

In [33]:
from sklearn.model_selection import train_test_split

train, test = train_test_split()

('Similar to other strategy type games like Agricola and others, it is a fun game with a decent amount of strategy involved.  I would definitely recommend for strategy game players particularly if you have a D&D Background (though that is not required)',
 4.0)